In [ ]:
'''
1.Get all the XGBoost models
2.Get all the LSTM models
'''

In [1]:
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import XGBClassifier as XGBC
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split as TTS
from sklearn.metrics import confusion_matrix as cm, recall_score as recall, roc_auc_score as auc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score as accuracy, recall_score as recall,roc_auc_score as auc
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
import numpy as np
from matplotlib import pyplot as plt
from xgboost import plot_importance
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Read files from colab
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


['installbackupandsync.exe',
 'sample_submission.csv',
 'test_clean_kalman.csv',
 'train_clean_kalman.csv',
 'open-data-fin.csv',
 'Colab Notebooks',
 'Andrew_tensorflow_leeson',
 'newtrain.csv',
 'newtrain2.csv',
 'newtrain3.csv',
 'datawithpr_withtrainid.csv',
 'datawithpr_havepr.csv',
 'datapr1.0.csv',
 'datapr_planC.csv',
 'planC_with_state.csv',
 'full_data_with_state.csv',
 'T1K11_2.csv',
 'save',
 'Model',
 'LSTM',
 'lstm_model.h5',
 'new_clear_data.csv',
 'pdfullindex.csv',
 'datawithstate.csv',
 '120000_pd_pr_train_id.csv',
 '150000_pd_pr_train_id.csv',
 '180000_pd_pr_train_id.csv',
 '210000_pd_pr_train_id.csv',
 '240000_pd_pr_train_id.csv',
 '270000_pd_pr_train_id.csv',
 '300000_pd_pr_train_id.csv',
 'pd_full_pr_train_id.csv',
 'newderby_datawithca0.csv',
 'lstm_fullpred_model.h5',
 'new_testdata_with_ca.csv',
 'full_testdata_with_ca_state.csv',
 'open-data-2-derby.csv',
 '1M00_softmax.h5',
 '1S41_simpleRNN.h5',
 'lstm_fullpred_model_vs.h5',
 'simpleRNN_fullpred_model_vs.h5']

In [2]:
!pip install tensorflow_addons
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Input, Dense, Add, Multiply
import pandas as pd
import numpy as np
import random
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa
import gc

from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score


In [3]:
#read data from dataset
data = pd.read_csv('full_data_with_state.csv',index_col= 0)
data

,Timestamp,Type,ID,Event2,State,Panel_Request,From_Berth,To_Berth,Train_ID,SDY544,SDY551,SDY552,SDY555,SDY556,SDY557,SDY559,SDY562,SDY564,SDY569,SDY571,SDY572,SDY574,SDY581,SDY583,STD5027,STD5029,STD5030,STD5032,STD5033,STD5035,STD5036,STD5037,STD5038,STD5039,STD5040,STD5043,STD5044,STD5045,STD5046,STD5049,...,SDW5302,SDW5304,SDW5306,SDW5308,SDW5309,SDW5310,SDW5311,SDW5313,SDW5315,SDW5316,SDW5318,SDW5319,SDW5321,SDW5322,SDW5324,SDW5328,SDW5329,SDW5330,SDW5331,SDW5332,SDW5333,SEC5474,SEC5475,SEC5477,SEC5480,SEC5481,SEC5483,SEC5484,SEC5486,SEC5487,SEC5488,SEC5489,SEC5490,SEC5491,SEC5493,SEC5496,SEC5501,SEC5502,SEC5503,SEC5504
1,18/10/2019 10:42,Signal,STD5053,STD5053,0,NaN,NaN,NaN,NaN,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,18/10/2019 10:42,CA,B5313,CAB5053B5313,1K11,NaN,B5053,B5313,1K11,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,18/10/2019 10:42,CA,B0556,CAB0554B0556,1F20,NaN,B0554,B0556,1F20,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,18/10/2019 10:42,Route,RDC5065C(M),RDC5065C(M),0,NaN,NaN,NaN,NaN,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,18/10/2019 10:42,Signal,STD5032,STD5032,1,NaN,NaN,NaN,NaN,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991449,08/11/2019 13:31,Panel Request,RDW5306A(C),RDW5306A(C),1,NaN,NaN,NaN,NaN,1,0,1,0,1,1,0,0,1,0,0,0,1,0,1,0,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
991453,08/11/2019 13:31,Signal,STD5029,STD5029,0,NaN,NaN,NaN,NaN,1,0,1,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
991455,08/11/2019 13:31,CA,B5033,CAB5029B5033,1V10,QRTD5045B-2(M),B5029,B5033,1V10,1,0,1,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
991456,08/11/2019 13:31,Panel Request,RTD5045B-2(M),RTD5045B-2(M),1,NaN,NaN,NaN,NaN,1,0,1,0,1,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
#Get trains with more than 400 lines of data
def cal_train_number(data):
    tid=data['Train_ID'].unique().tolist()
    tid.remove(np.nan)
    tid_withlen=[]
    each=[]
    for i in tid:
        each=[i,len(data[data['Train_ID']==i])]
        tid_withlen.append(each)
   
    def takeSecond(elem):
        return elem[1]
    
    tid_withlen.sort(key=takeSecond,reverse=True)
    return tid_withlen


ctn = cal_train_number(data)
train_idlist=[]
for i in ctn:
    if i[1]>400:
        train_idlist.append(i[0])
train_idlist

In [ ]:
'''
1.Get all the XGBoost models
'''

In [ ]:
#predict what the corresponding panel request is for each train 
#save the XGBoost model for each train

In [6]:
# Take out the ID with route
def get_ID_withroute(ID_data):
    new_ID=[]
    for i in ID_data:
        if i[0]=='B':
            new_ID.append(i[1:5])
        elif i[0:3]=='RDY' :
            new_ID.append('0'+i[3:])
        elif i[0]=='R':
            new_ID.append(i[3:])
        elif i[0:3]=='SDY':
            new_ID.append('0'+i[3:6])
        elif i[0]=='S':
            new_ID.append(i[3:7])
    return new_ID

In [7]:
# Take out the ID
def get_ID(ID_data):
    new_ID_2=[]
    for i in ID_data:
            if i[0]=='B':
                new_ID_2.append(i[1:5])
            elif i[0:3]=='RDY' :
                new_ID_2.append('0'+i[3:6])
            elif i[0]=='R':
                new_ID_2.append(i[3:7])
            elif i[0:3]=='SDY':
                new_ID_2.append('0'+i[3:6])
            elif i[0]=='S':
                new_ID_2.append(i[3:7])
    return new_ID_2

In [8]:
# Find out what kinds of panel requests there are
def get_unique_pr_with0(train_ID,data):
    train1=data[data['Train_ID']==train_ID]
    new_ID=get_ID_withroute(train1['ID'].tolist()) 

    type1= train1['Type'].tolist()
    pr_type=[]

    for i in range(0,len(type1)):
        if type1[i]=='Panel Request':
            pr_type.append(new_ID[i])
        else:
            pr_type.append(0)

    pr_unique=[]
    for j in pr_type:
        if (j != 0)and (j not in pr_unique):
            pr_unique.append(j)

    return pr_unique,pr_type

In [9]:
# Define a function to get test data for a train
def get_train_message(train_ID,data):
    train1=data[data['Train_ID']==train_ID]
    train1['Timestamp']=range(1,len(train1)+1)
    
    new_ID_2=get_ID(train1['ID'].tolist())
    train1['ID']=new_ID_2
    
    train1.drop(['State','Panel_Request','From_Berth','To_Berth'],inplace=True,axis=1)  
    pr_unique,pr_type=get_unique_pr_with0(train_ID,data)

    pr_type_list=[]
    for i in pr_type:
        if i in pr_unique:
            pr_type_list.append(pr_unique.index(i)+1)
        else:
            pr_type_list.append(0)
        
    train1.drop(['Event2'],inplace=True,axis=1)
    train1['Type']=pr_type_list
    train1.index=range(1,len(train1)+1)
    
    

    #One-hot encoding of Signal ID
    from sklearn.preprocessing import OneHotEncoder
    
    X = train1.iloc[:,[2]]
    enc = OneHotEncoder(categories='auto').fit(X)
    result = enc.transform(X).toarray()
    signalid=[]
    for i in enc.get_feature_names():
        signalid.append(i[3:])
    result = pd.DataFrame(result)
    result.index=range(1,len(result)+1)
    result.columns = signalid
    train1 = pd.concat([train1,result],axis=1)   
    
    train1.drop(['ID','Timestamp'],inplace=True,axis=1)


    #One-hot encoding of train ID
    X = train1.iloc[:,[1]]
    enc = OneHotEncoder(categories='auto').fit(X)
    result = enc.transform(X).toarray()
    signalid=[]
    for i in enc.get_feature_names():
        signalid.append(i[3:])
    result = pd.DataFrame(result)
    result.index=range(1,len(result)+1)
    result.columns = signalid
    train1 = pd.concat([train1,result],axis=1)  

    train1.drop(['Train_ID'],inplace=True,axis=1)
    
    return train1

In [10]:
train_message=get_train_message('1S41',data)
train_message

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,Type,SDY544,SDY551,SDY552,SDY555,SDY556,SDY557,SDY559,SDY562,SDY564,SDY569,SDY571,SDY572,SDY574,SDY581,SDY583,STD5027,STD5029,STD5030,STD5032,STD5033,STD5035,STD5036,STD5037,STD5038,STD5039,STD5040,STD5043,STD5044,STD5045,STD5046,STD5049,STD5053,STD5054,STD5055,STD5056,SDC5061,SDC5062,SDC5063,SDC5064,...,SEC5483,SEC5484,SEC5486,SEC5487,SEC5488,SEC5489,SEC5490,SEC5491,SEC5493,SEC5496,SEC5501,SEC5502,SEC5503,SEC5504,0544,0552,0554,0556,0564,0574,0584,0586,5061,5075,5083,5093,5101,5109,5111,5300,5316,5324,5328,5330,5332,B305,C833,DMLS,X061,1S41
1,1,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,1,1,1,1,1,1,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,1,1,1,1,1,0,0,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1568,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1569,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1570,0,1,1,1,0,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
#Get the XGBoost model for all trains with more than 400 lines of data
for i in range(0,len(train_idlist)):
    train_message=get_train_message(train_idlist[i],data)    #Call get_train_message to get the data of a train
    
    #Model training for the data obtained above
    data1=train_message.copy()
    data1=data1[data1['Type']!=0]
    X = data1.iloc[:,data1.columns != "Type"]
    y = data1.iloc[:,data1.columns == "Type"]
    from sklearn.model_selection import train_test_split
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3,random_state=513)
    for j in [Xtrain, Xtest, Ytrain, Ytest]:
        j.index = range(j.shape[0])
    
    #Use the parameters obtained in XGBoost_vs_other
    clf = XGBC(max_depth=3, 
      learning_rate=0.1, 
      n_estimators=100,
      objective='multi:softmax', 
      booster='gbtree', 
      gamma=0,
      subsample=0.8,
      ).fit(Xtrain, Ytrain)
    
    #Calling joblib to store the XGBoost model
    from sklearn.externals import joblib
    joblib.dump(clf, 'save/'+train_idlist[i]+'_real_pr.pkl') 

流式输出内容被截断，只能显示最后 5000 行内容。
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [ ]:
'''
2.Get all the LSTM models
'''

In [ ]:
#LSTM modeling and storage

In [ ]:
#Integrate functions to get all LSTM models.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        keras.layers.LSTM(units = rnn_units,
                          stateful = True,
                          recurrent_initializer = 'glorot_uniform',
                          return_sequences = True),

       # keras.layers.Dense(vocab_size, activation='softmax')

        keras.layers.Dense(vocab_size),
    ])
    return model



def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

def split_input_target(id_text):
    return id_text[0:-1],id_text[1:]

In [ ]:
# Use this function to get the LSTM model for a given train
def get_all_lstm_model(train_ID,data):

    data1=data[data['Train_ID']==train_ID]
    timestamp=[]
    for i in data1['Timestamp'].tolist():
        timestamp.append(i[0:2])
    data1['Timestamp']=timestamp


    data1=data1.iloc[:,0:4]
    data1.drop('ID',inplace=True,axis=1)
    data1.index= range(0,len(data1))

    event2=data1['Event2'].tolist()
    new_event=[]
    for i in event2:
        if i[0:3]=='RDY':
            new_event.append('RDY0'+i[3:6])
        elif i[0]=='R':
            new_event.append(i[0:7])
        else:
            new_event.append(i)
    data1['Event2']=new_event
    data1 = data1[data1['Type'].isin(['CA','Panel Request'])]

    #Use the types of events that may occur in the dataset as a dictionary
    dict_=data1['Event2'].unique().tolist()
    idx2char = np.array(dict_)

    timestamp_un=[]
    for i in timestamp:
        if i not in timestamp_un:
            timestamp_un.append(i)

    char2idx={char:idx for idx,char in enumerate(dict_)}
    
    #Possible events in the dataset as a sequence of events
    all_event=data1['Event2'].tolist()
    event_as_int = np.array([char2idx[c] for c in all_event])

    char_dataset = tf.data.Dataset.from_tensor_slices(event_as_int)
    seq_length = 20 
    seq_dataset = char_dataset.batch(seq_length+1,drop_remainder = True)
    seq_dataset = seq_dataset.map(split_input_target)

    #According to the different data amount of each train, set the corresponding batch_size
    if int(len(data1)/200)*8 ==0:
        bs=8
    else:
        bs=int(len(data1)/200)*8
    batch_size = bs

    buffer_size = 10000
    seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

    vocab_size = len(dict_)
    embedding_dim = 256
    rnn_units = 1024

    model = build_model(
        vocab_size = vocab_size,
        embedding_dim = embedding_dim,
        rnn_units = rnn_units,
        batch_size = batch_size)

    model.compile(optimizer = 'adam', loss = loss)
    
    epochs = 200
    model.fit(seq_dataset, epochs = epochs)

    model.save('LSTM/'+train_ID+'_lstm_model.h5')

    return

In [ ]:
# Obtain LSTM models for all trains with more than 400 lines of data
fullacc_list=[]
counts=0
for i in train_idlist:
    counts=counts+1
    print(counts,i)
    get_all_lstm_model(i,data) 

1 1M00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


流式输出内容被截断，只能显示最后 5000 行内容。
1/1 [==============================] - 0s 874us/step - loss: 0.2026
Epoch 108/200
1/1 [==============================] - 0s 911us/step - loss: 0.2404
Epoch 109/200
1/1 [==============================] - 0s 969us/step - loss: 0.2040
Epoch 110/200
1/1 [==============================] - 0s 954us/step - loss: 0.2304
Epoch 111/200
1/1 [==============================] - 0s 854us/step - loss: 0.1590
Epoch 112/200
1/1 [==============================] - 0s 853us/step - loss: 0.1999
Epoch 113/200
1/1 [==============================] - 0s 905us/step - loss: 0.1978
Epoch 114/200
1/1 [==============================] - 0s 1ms/step - loss: 0.2075
Epoch 115/200
1/1 [==============================] - 0s 922us/step - loss: 0.1932
Epoch 116/200
1/1 [==============================] - 0s 1ms/step - loss: 0.1452
Epoch 117/200
1/1 [==============================] - 0s 854us/step - loss: 0.1863
Epoch 118/200
1/1 [==============================] - 0s 1ms/step - loss: 0.1769
Epoch 119